## TF-IDF + Classifiers

In [66]:
import pandas as pd
df = pd.read_json("./data/SubtaskA.jsonl", lines=True)
print(len(df))

152809


In [67]:
df.label.value_counts() # 1 - machine-generated

1    87632
0    65177
Name: label, dtype: int64

In [68]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.label, test_size=0.3, random_state=38)

In [69]:
X_train_text = X_train
X_test_text = X_test

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(X_train)

In [5]:
X_test = vectorizer.transform(X_test)

## Logreg

In [6]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=38)

clf.fit(X_train, y_train)

LogisticRegression(random_state=38)

In [7]:
y_pred = clf.predict(X_test)

In [8]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86     19574
           1       0.89      0.91      0.90     26269

    accuracy                           0.88     45843
   macro avg       0.88      0.88      0.88     45843
weighted avg       0.88      0.88      0.88     45843



## Boosting

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

In [20]:
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0,
    max_depth=2, random_state=0, verbose=True).fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.2031           69.68m
         2           1.1433           68.45m
         3           1.0764           67.87m
         4           1.0352           67.66m
         5           0.9958           67.48m
         6           0.9710           67.76m
         7           0.9407           67.54m
         8           0.9152           67.36m
         9           0.9004           67.24m
        10           0.8855           67.15m
        20           0.7845           66.43m
        30           0.7255           65.63m
        40           0.6849           64.89m
        50           0.6488           64.18m
        60           0.6213           63.48m
        70           0.6006           62.75m
        80           0.5831           62.13m
        90           0.5677           61.46m
       100           0.5508           60.77m
       200           0.4566           53.94m
       300           0.4038           47.08m
       40

In [21]:
y_pred = clf.predict(X_test)

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88     19574
           1       0.90      0.92      0.91     26269

    accuracy                           0.89     45843
   macro avg       0.89      0.89      0.89     45843
weighted avg       0.89      0.89      0.89     45843



## SVC

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
clf = SVC(gamma='auto', verbose=True)

In [34]:
X_train.shape

(106966, 5000)

In [36]:
clf.fit(X_train[0:10000], y_train[0:10000])

/tmp/ipykernel_1115798/1894818839.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  clf.fit(X_train[0:10000], y_train[0:10000])


[LibSVM]....
*
optimization finished, #iter = 4899
obj = -8657.820835, rho = 1.008493
nSV = 8676, nBSV = 8676
Total nSV = 8676


SVC(gamma='auto', verbose=True)

In [37]:
y_pred = clf.predict(X_test)

In [39]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     19574
           1       0.57      1.00      0.73     26269

    accuracy                           0.57     45843
   macro avg       0.29      0.50      0.36     45843
weighted avg       0.33      0.57      0.42     45843



/home/user/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/user/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Word2Vec

In [56]:
# from gensim.models import Word2Vec

# sentences = [sentence.split() for sentence in X_train_text]
# w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

In [58]:
# w2v_model.save("word2vec.model")

In [70]:
from gensim.models import Word2Vec

model = Word2Vec.load("word2vec.model")

In [71]:
import numpy as np

def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_vector = np.array([vectorize(sentence) for sentence in X_train_text])
X_test_vector = np.array([vectorize(sentence) for sentence in X_test_text])

## Logreg

In [72]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_vector, y_train)

/home/user/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [73]:
y_pred = clf.predict(X_test_vector)

In [74]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.75      0.75     19574
           1       0.82      0.82      0.82     26269

    accuracy                           0.79     45843
   macro avg       0.79      0.79      0.79     45843
weighted avg       0.79      0.79      0.79     45843



## Boosting

In [81]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0, verbose=True).fit(X_train_vector, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.3038            2.58m
         2           1.2617            2.55m
         3           1.2272            2.52m
         4           1.2004            2.49m
         5           1.1740            2.47m
         6           1.1508            2.44m
         7           1.1341            2.41m
         8           1.1190            2.39m
         9           1.1041            2.36m
        10           1.0892            2.33m
        20           0.9954            2.07m
        30           0.9470            1.81m
        40           0.9103            1.56m
        50           0.8839            1.30m
        60           0.8607            1.04m
        70           0.8419           46.64s
        80           0.8274           31.09s
        90           0.8138           15.54s
       100           0.8032            0.00s


In [83]:
y_pred = clf.predict(X_test_vector)

In [84]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.78      0.78     19574
           1       0.83      0.83      0.83     26269

    accuracy                           0.81     45843
   macro avg       0.80      0.80      0.80     45843
weighted avg       0.81      0.81      0.81     45843



## KNN

In [89]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)

In [90]:
clf.fit(X_train_vector, y_train)

KNeighborsClassifier()

In [91]:
y_pred = clf.predict(X_test_vector)

In [92]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.92      0.82     19574
           1       0.93      0.77      0.84     26269

    accuracy                           0.83     45843
   macro avg       0.84      0.84      0.83     45843
weighted avg       0.85      0.83      0.83     45843



## SVC

In [93]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
clf = SVC(gamma='auto', verbose=True)

In [95]:
clf.fit(X_train_vector[0:2000], y_train[0:2000])

[LibSVM]*
optimization finished, #iter = 955
obj = -1227.250966, rho = 6.655179
nSV = 1446, nBSV = 1402
Total nSV = 1446


/tmp/ipykernel_1115798/1688042585.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  clf.fit(X_train_vector[0:2000], y_train[0:2000])


SVC(gamma='auto', verbose=True)

In [96]:
y_pred = clf.predict(X_test_vector)

In [97]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77     19574
           1       0.85      0.77      0.81     26269

    accuracy                           0.79     45843
   macro avg       0.79      0.79      0.79     45843
weighted avg       0.80      0.79      0.79     45843



## Glove

In [98]:
# First we will prepare our train dataset

In [110]:
corpus = X_train_texts.apply(lambda x: x.replace('\n', ' '))
corpus = X_train_texts.apply(lambda x: x.replace('<unk>', '')).values

In [111]:
corpus = '\n'.join(list(corpus))

In [112]:
len(corpus)

271594941

In [113]:
with open('train_corpus.txt', "w+") as f:
    f.write(corpus)

In [124]:
# I performed the training
result_path = "GloVe/vectors.txt"

In [128]:
# Load GloVe embeddings into a dictionary
def load_embeddings(file_path):
    embeddings = {}
    bad_values = 0
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:], dtype='float32')
                embeddings[word] = vector
            except ValueError:
                bad_values += 1
    print(bad_values)
    return embeddings

glove_embeddings_path = result_path  # Adjust the path to your downloaded GloVe file
glove_embeddings = load_embeddings(glove_embeddings_path)

186


In [130]:
len(glove_embeddings)

214677

In [175]:
# for some reason the dimensions for those values are incorrect, lets just remove them
counter  = 0
for i in m:
    if len(i) != 50:
        counter +=1
print(counter)

13


In [181]:
from copy import deepcopy

ite = deepcopy(glove_embeddings)

In [188]:
from copy import copy
for name, i in ite.items():
    if len(i) != 50:
        glove_embeddings.pop(name)

In [190]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [glove_embeddings[word] for word in words if word in glove_embeddings]
    if len(words_vecs) == 0:
        return np.zeros(50)
    words_vecs = np.array(words_vecs, dtype='object')
    return words_vecs.mean(axis=0)

X_train_vector = np.array([vectorize(sentence) for sentence in X_train_text])
X_test_vector = np.array([vectorize(sentence) for sentence in X_test_text])

In [191]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_vector, y_train)

/home/user/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [192]:
y_pred = clf.predict(X_test_vector)

In [193]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.68      0.70     19574
           1       0.77      0.80      0.78     26269

    accuracy                           0.75     45843
   macro avg       0.74      0.74      0.74     45843
weighted avg       0.75      0.75      0.75     45843



## Boosting

In [194]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
    max_depth=1, random_state=0, verbose=True).fit(X_train_vector, y_train)

      Iter       Train Loss   Remaining Time 
         1           1.2885            1.34m
         2           1.2464            1.30m
         3           1.2240            1.28m
         4           1.2004            1.27m
         5           1.1827            1.25m
         6           1.1713            1.24m
         7           1.1544            1.23m
         8           1.1438            1.21m
         9           1.1347            1.20m
        10           1.1240            1.18m
        20           1.0571            1.05m
        30           1.0218           54.86s
        40           0.9913           46.92s
        50           0.9655           39.03s
        60           0.9462           31.19s
        70           0.9316           23.37s
        80           0.9180           15.58s
        90           0.9077            7.79s
       100           0.8974            0.00s


In [195]:
y_pred = clf.predict(X_test_vector)

In [196]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.74      0.74     19574
           1       0.81      0.81      0.81     26269

    accuracy                           0.78     45843
   macro avg       0.77      0.77      0.77     45843
weighted avg       0.78      0.78      0.78     45843



## KNN

In [197]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)

In [199]:
clf.fit(X_train_vector, y_train)

KNeighborsClassifier()

In [200]:
y_pred = clf.predict(X_test_vector)

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7fa3d7763790>
Traceback (most recent call last):
  File "/home/user/conda/lib/python3.9/site-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/home/user/conda/lib/python3.9/site-packages/threadpoolctl.py", line 1175, in _make_controller_from_path
    lib_controller = controller_class(
  File "/home/user/conda/lib/python3.9/site-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
  File "/home/user/conda/lib/python3.9/ctypes/__init__.py", line 382, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: dlopen() error


In [201]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.92      0.82     19574
           1       0.93      0.75      0.83     26269

    accuracy                           0.82     45843
   macro avg       0.83      0.84      0.82     45843
weighted avg       0.84      0.82      0.82     45843



## SVC

In [202]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
clf = SVC(gamma='auto', verbose=True)

In [203]:
clf.fit(X_train_vector, y_train)

[LibSVM]...................................
*.
*
optimization finished, #iter = 36232
obj = -59137.966641, rho = 49.734867
nSV = 64282, nBSV = 64211
Total nSV = 64282


KeyboardInterrupt: 

In [ ]:
y_pred = clf.predict(X_test_vector)

In [ ]:
print(classification_report(y_test, y_pred))